In [ ]:
#01 farneback

In [3]:
##este ejercicio no

import cv2
import numpy as np
cap = cv2.VideoCapture("videos/slow_traffic_small.mp4")
 
ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
 
while(1):
  ret, frame2 = cap.read()
  next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
 
  flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
 
  mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
  hsv[...,0] = ang*180/np.pi/2
  hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
  rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
 
  cv2.imshow('frame2',rgb)
  k = cv2.waitKey(30) 
  if k == 27:
    break
  elif k == ord('s'):
    cv2.imwrite('opticalfb.png',frame2)
    cv2.imwrite('opticalhsv.png',rgb)
  prvs = next
 
cap.release()
cv2.destroyAllWindows()

In [ ]:
#02 flujo optico lukas Kanade

In [2]:

import numpy as np
import cv2
 
cap = cv2.VideoCapture('videos/slow_traffic_small.mp4')
# parametros para detección de esquinas ShiTomasi
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
 
# Parámetros para el flujo óptico de Lucas Kanade
lk_params = dict( winSize = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
 
# Crea algunos colores aleatorios
color = np.random.randint(0,255,(100,3))
# Toma el primer cuadro y encuentra esquinas en él
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
 
# Crear una máscara de imagen para dibujar
mask = np.zeros_like(old_frame)
 
while(1):
  ret,frame = cap.read()
  frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
  # calcula optical flow
  p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
 
  # Select good points
  good_new = p1[st==1]
  good_old = p0[st==1]
 
  # dibuja las lineas
  for i,(new,old) in enumerate(zip(good_new,good_old)):
    a,b = new.ravel()
    c,d = old.ravel()
    mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
    frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
  img = cv2.add(frame,mask)
 
  cv2.imshow('frame',img)
  k = cv2.waitKey(30) 
  if k == 27:
    break
 
  # Ahora actualiza el marco anterior y los puntos anteriores
  old_gray = frame_gray.copy()
  p0 = good_new.reshape(-1,1,2)
 
cv2.destroyAllWindows()
cap.release()

<ipython-input-2-27f49a6bb714>:41: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
<ipython-input-2-27f49a6bb714>:42: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)


In [ ]:
#03 mean shift1

In [1]:
import numpy as np
import cv2

cap = cv2.VideoCapture('videos/slow_traffic_small.mp4')

# tomar el primer fotograma del video
ret,frame = cap.read()

# configurar la ubicación inicial de la ventana
# r,h,c,w - region de la imagen
r,h,c,w = 200,20,300,20  
track_window = (c,r,w,h)

# configurar el ROI para el seguimiento
roi = frame[r:r+h, c:c+w]
hsv_roi =  cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Configure los criterios de terminación, ya sea 10 iteraciones o mueva al menos 1 punto
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while(1):
    ret ,frame = cap.read()

    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # aplicar meanshift para obtener la nueva ubicación
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        # Dibujarlo en la imagen
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        cv2.imshow('img2',img2)

        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
        else:
            cv2.imwrite(chr(k)+".jpg",img2)

    else:
        break

cv2.destroyAllWindows()
cap.release()

In [ ]:
#05 optical flow1

In [ ]:
import numpy as np
import cv2 as cv

cap = cv.VideoCapture('../videos/soccer.mp4')

# parámetros para la detección de esquinas ShiTomasi
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parámetros del flujo óptico lucas kanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Crea algunos colores aleatorios
color = np.random.randint(0,255,(100,3))

# Toma el primer fotograma y encuentra esquinas en él.
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Cree una imagen de máscara para fines de dibujo
mask = np.zeros_like(old_frame)

while(1):
    ret,frame = cap.read()
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # calcular el flujo óptico
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Seleccione buenos puntos
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]

    # dibujar las pistas
    for i,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv.line(mask, (int(a),int(b)),(int(c),int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame,(int(a),int(b)),5,color[i].tolist(),-1)
    img = cv.add(frame,mask)

    cv.imshow('frame',img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

    # Ahora actualice el cuadro anterior y los puntos anteriores
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

In [ ]:
#optical flow2

In [ ]:
import cv2 as cv 
import numpy as np 
cap = cv.VideoCapture("../videos/slow_traffic_small.mp4") 
ret, first_frame = cap.read() 
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY) 
mask = np.zeros_like(first_frame) 
mask[..., 1] = 255
while(cap.isOpened()): 
    ret, frame = cap.read() 
    cv.imshow("input", frame) 
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY) 
    flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0) 
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1]) 
    mask[..., 0] = angle * 180 / np.pi / 2
    mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX) 
    rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR) 
    cv.imshow("dense optical flow", rgb) 
    prev_gray = gray 
    if cv.waitKey(1) & 0xFF == ord('q'): 
        break
  
cap.release() 
cv.destroyAllWindows()

In [ ]:
#04 opencv tracker

In [6]:
# Importar modulos
import cv2
import numpy as np
# Crear un objeto VideoCapture
video = cv2.VideoCapture("videos/people.mp4")
# Check if video opened successfully
if video.isOpened() == False:
    print("Could not open video!")

# Leer el primer cuadro
ret, frame = video.read()
# Compruebe si el cuadro se lee correctamente
if ret == False:
    print("Cannot read video")  
# Muestra el primer fotogramae
cv2.imshow("First Frame",frame)
cv2.waitKey(0)
cv2.imwrite("firstFrame.png", frame)
# Especificar el cuadro delimitador inicial
bbox = cv2.selectROI(frame)
print(bbox)
cv2.destroyAllWindows()

# MedianFlow Tracker
#tracker = cv2.legacy.TrackerMedianFlow_create()
tracker = cv2.TrackerMIL_create()
#tracker = cv2.legacy.TrackerMOSSE_create()
# Inicializar rastreador
ret = tracker.init(frame,bbox)
# Crea una nueva ventana donde mostraremos
# los resultados
cv2.namedWindow("Tracker")
#Mostrar el primer cuadro
cv2.imshow("Tracker",frame)
while True:
    # Leer siguiente fotograma
    ret, frame = video.read()
    # Compruebe si se leyó el cuadro
    if ret == False:
        break
    # Actualizar rastreador
    found, bbox = tracker.update(frame)
    # Si se encuentra el objeto, dibuja bbox
    if found:
        # Esquina superior izquierda
        topLeft = (int(bbox[0]), int(bbox[1]))
        # Esquina inferior derecha
        bottomRight = (int(bbox[0]+bbox[2]),int(bbox[1]+bbox[3]))
        # Mostrar cuadro delimitador
        cv2.rectangle(frame, topLeft, bottomRight, (0,0,255), 2)
    else:
        # Mostrar estado
        cv2.putText(frame, "Object not found", (20,70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 2)
    # Mostrar cuadro
    cv2.imshow("Tracker",frame)
    k = cv2.waitKey(5)
    if k == 27:
        break
cv2.destroyAllWindows()

(93, 23, 67, 61)


In [ ]:
#tracker 2

In [12]:

import cv2
import sys

(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

if __name__ == '__main__' :

    # Set up tracker.
    # Instead of MIL, you can also use

    tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'CSRT']
    tracker_type = tracker_types[1]

    if int(minor_ver) < 3:
        tracker = cv2.Tracker_create(tracker_type)
    else:
        if tracker_type == 'BOOSTING':
            tracker = cv2.TrackerBoosting_create()
        if tracker_type == 'MIL':
            tracker = cv2.TrackerMIL_create()
        if tracker_type == 'KCF':
            tracker = cv2.TrackerKCF_create()
        if tracker_type == 'TLD':
            tracker = cv2.TrackerTLD_create()
        if tracker_type == 'MEDIANFLOW':
            tracker = cv2.legacy.TrackerMedianFlow_create()
        if tracker_type == 'GOTURN':
            tracker = cv2.TrackerGOTURN_create()
        if tracker_type == "CSRT":
            tracker = cv2.legacy.TrackerCSRT_create()

    # Read video
    video = cv2.VideoCapture("videos/slow_traffic_small.mp4")

    # Exit if video not opened.
    if not video.isOpened():
        print ("Could not open video")
        sys.exit()

    # Read first frame.
    ok, frame = video.read()
    if not ok:
        print ('Cannot read video file')
        sys.exit()
    
    # Define an initial bounding box
    bbox = (287, 23, 86, 320)

    # Uncomment the line below to select a different bounding box
    bbox = cv2.selectROI(frame, False)

    # Initialize tracker with first frame and bounding box
    ok = tracker.init(frame, bbox)

    while True:
        # Read a new frame
        ok, frame = video.read()
        if not ok:
            break
        
        # Start timer
        timer = cv2.getTickCount()

        # Update tracker
        ok, bbox = tracker.update(frame)

        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);

        # Draw bounding box
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

        # Display tracker type on frame
        cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255),2);
    
        # Display FPS on frame
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);


        # Display result
        cv2.imshow("Tracking", frame)

        # Exit if ESC pressed
        k = cv2.waitKey(1) & 0xff
        if k == 27 : 
            break

AttributeError: module 'cv2' has no attribute 'Tracker_create'